In [ ]:
import neuroglancer.static_file_server
from pytools.ng.viz import add_zarr_image, add_roi_annotations
from pytools import HedwigZarrImages
from pathlib import Path

%matplotlib inline
import matplotlib.pyplot as plt

import SimpleITK as sitk
import json

In [ ]:
file_dir = Path("/Users/blowekamp/scratch/hedwig/Xenium/")
server = neuroglancer.static_file_server.StaticFileServer(
        static_dir=file_dir, bind_address="localhost", daemon=True
    )
viewer = neuroglancer.Viewer()

In [ ]:
with viewer.txn() as s:
  dimensions = neuroglancer.CoordinateSpace( names=["x", "y", "z"], units =["µm","µm", "µm"], scales=[1,1,1])
  s.dimensions = dimensions
  s.layout = neuroglancer.DataPanelLayout("xy")
  s.show_axis_lines = False
  s.show_default_annotations = False  
  
  s.layers.clear()

  i=0  
  while True:
    fn =   file_dir/"2025_05_27_Female002.zarr"/f"{i}"
    if fn.exists():
      print(f"Adding {fn}...")
      add_zarr_image(s, fn, server_url=server.url)
    else:
      break
    i += 1    

print(viewer)

In [ ]:
with viewer.txn() as s:
  dimensions = neuroglancer.CoordinateSpace( names=["x", "y", "z"], units =["µm","µm", "µm"], scales=[1,1,1])
  s.dimensions = dimensions
  s.layout = neuroglancer.DataPanelLayout("xy")
  s.show_axis_lines = False
  s.show_default_annotations = False  
  
  s.layers.clear()

  i=0  
  while True:
    fn =   file_dir/"2025_05_14_0437_UF_DAPI_20X_Female.zarr"/f"{i}"
    if fn.exists():
      print(f"Adding {fn}...")
      add_zarr_image(s, fn, server_url=server.url)
    else:
      break
    i += 1    

print(viewer)

In [ ]:
def describe_zarr_ome_xml(fn: Path):
    hwz_images = HedwigZarrImages(fn)
    ome_info = hwz_images.ome_info
    for image_index in range(ome_info.number_of_images()):
        image_et = ome_info._image_element(image_index)
        
        print(f"Name: \"{list(ome_info.image_names())[image_index]}\"")
        el = ome_info._image_element(image_index).iterfind("OME:Pixels/OME:Channel", ome_info._ome_ns)
        
        for i, e in enumerate(el):
            channel_line = f"\tChannel: {i}"
            for a in ["Name", "Flour", "IlluminationType"]:
                if a in e.attrib:
                    channel_line += f" {a}: \"{e.attrib[a]}\""
            print(channel_line)        
        print(f"\tDimension Order: {ome_info.dimension_order(image_index)}")
        print(f"\tSize: {ome_info.size(image_index)}")
        print(f"\tSpacing/PhysicalSize: {ome_info.size(image_index)}")
        print(f"\tType: {image_et.find('OME:Pixels', ome_info._ome_ns).attrib['Type']}")
        print(f"\tShader Type: {hwz_images[image_index].shader_type}")

    return hwz_images
        

In [ ]:
hwz_images = describe_zarr_ome_xml(file_dir/"2025_05_27_Female002.zarr")
if "label image" in hwz_images. get_series_keys():
    sitk_image = hwz_images["label image"].extract_2d(512,512, auto_uint8=True)
    # Rotate the image 90 CW
    sitk_image = sitk.Flip(sitk.PermuteAxes(sitk_image, [1, 0]), [True, False])
    plt.imshow(sitk.GetArrayViewFromImage(sitk_image))

for hw_image in hwz_images:
    print(hw_image.neuroglancer_shader_parameters(middle_quantile=[0.04, 1.0] ))


In [ ]:
fn = file_dir/"2025_05_14_0437_UF_DAPI_20X_Female.zarr"

hwz_images = describe_zarr_ome_xml(fn)
for name in hwz_images.get_series_keys():
    params = hwz_images[name].neuroglancer_shader_parameters(middle_quantile=[0.04, 1.0] )
    # save each params to the fn name but _{name}.json instead of .zarr
    with open(fn.with_name(fn.name + f"_{name}.json"), "w") as f:
        json.dump(params, f, indent=2)